In [1]:
import numpy as np
import math
import random

In [2]:
#Initial Pheromone 
def FirstPheromone(hobs):
    return [[1/hobs for i in range(hobs)] for j in range(hobs)]

# A random graph
def Graph(dim):
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([0,1]))
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Eliminate Loops 
def Deloop(x):
    l = len(x)

    for i in range(l):
        for j in range(i+1,l):
            j = l+i-j
            
            if j < len(x):          # list changes as loop is removed 
                if x[i] == x[j]:
                    # print("Delooping:  ", x)
                    [x.remove(k) for k in x[i+1:j+1]]
                    # print("Delooped to: ", x)
    return x

In [3]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

In [4]:
# Transition Probability
def TransitionP(connections, alpha, beta):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    total = sum( [ (i[0]**alpha)*(i[1]**beta) for i in connections])    # sum heuristic and pheromone info
    prob = [( (((i**alpha)*(j**beta))/total) , k) for i,j,k in connections ]  # probability of each hob
    
    return prob

# pheromone, distance and hob information
def DPH(phermatrix, graph, connections, preshob):
    """ DPH(Distance Pheromone hob)
        takes a pheromone matrix
        graph matrix 
        connections in graph to use 
        present hob in graph
        and return the pheromone, distance and hob [(pheromone, distance, hob), ... , (pheromone, distance, hob)]"""
    
    dph = []
    for i in range(len(connections)):
        if connections[i] > 0:
            dph.append( (phermatrix[preshob][i], 1/graph[preshob][i], i) )
            
    #dph = [(phermatrix[preshob][i], graph[preshob][i], i) for i in range(len(connections)) if connections[i] > 0 ]
    
    return dph

def Disconnect(visited, connections):
    """ Disconnects already visited hobs
    visited: already visited hobs
    connections: connection vector with 0: no connection x: length of connection"""
    
    for i in visited:
        connections[i] = 0
        #print(i, visited, connections)
    #print("DONE++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++DONE")
    return connections

In [5]:
def FindPath(phermatrix, graph, takeoff, alpha = 2, beta = 2):
    
    path = []
    path.append(takeoff)
    
    while len(path) != len(graph):
        
        Cons = graph[takeoff].copy()
        tabu = path.copy()
        Cons = Disconnect(tabu, Cons)
        HP = DPH(phermatrix, graph, Cons, takeoff)
        prob = TransitionP(HP, alpha, beta)
        takeoff = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob])
        
        path.append(takeoff)
        
    #Deloop(path)
    #print("HERE ") 
    return path

In [6]:
# calculate cost of each route
def Cost(Pop, Cmatrix):
    Cvector = []
    
    for i in range(len(Pop)):
        route = Pop[i]
        cost = 0
        
        for j in range(len(route)-1):
            cost += Cmatrix[route[j]][route[j+1]]
        cost += Cmatrix[route[0]][route[len(route)-1]]   
        Cvector.append(cost)
        
    return Cvector

# Sort Chromosomes from Best to Worst
def Sortpop(Pop, Cmatrix):
    cost = Cost(Pop, Cmatrix)
    ans = [ x for _,x in sorted(zip(cost, Pop))]
    #cost = [ _ for _,x in sorted(zip(cost, Pop))]  # need be to use the true cost
    return ans

In [72]:
# Update Pheromone in visited arcs

def UpPheromone(route, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix

def UpPheromoneNPL(route, matrix):
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += 1
        matrix[route[i+1]][route[i]] += 1

    return matrix

# Evaporate pheromone 

def EvPheromone(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > 0:
                matrix[i][j] = (1-rho)*matrix[i][j]
            else:
                matrix[i][j] = math.exp(-5)
    return matrix

def EvPheromoneNeg(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            matrix[i][j] = (1-rho)*matrix[i][j]
        
    return matrix

In [73]:
# Pheromone initialization for TSP

#random.shuffle() shuffles the original list
#random.sample() returns a new shuffled list of given number of elements needed

def TSPher(graph):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    pher = [[cost for i in range(n)] for j in range(n)]
    
    return pher

In [74]:
# Simple Ant Colony Optimiza

def ACO101(Ngraph, ants, alpha = 1, beta = 3, rho = 0.5, runs = 50, Pher = None):
    
    if Pher == None:
        #Pher = FirstPheromone(len(Ngraph))     # to one 
        Pher = TSPher(Ngraph)                   # with route cost

    #starthob = random.choice([i for i in range(len(Ngraph))])
    i = 0
    while i < runs:
        Ants = []
        #print(Gr)
        starthob = random.choice([i for i in range(len(Ngraph))])
        for j in range(ants):
            stof = FindPath(Pher, Ngraph, starthob, alpha, beta)
            Ants.append(stof)
   
        #Pheromone update
        for j in Ants:
            UpPheromone(j, Pher)
            
        #Pheromone update  
        #EvPheromone(Pher, rho)
        EvPheromoneNeg(Pher, rho)
        
        #C = Cost(Ants, Ngraph)
        i += 1 
        
    C = Cost(Ants, Ngraph)    
    print("Minimum Cost Found = ", min(C), "Maximum Cost Found = ", max(C))
    return np.array(Ants), np.array(Pher)

In [75]:
# for trial, randomly define cost matrix if not given. randomly generates a cost for a route in the range bound
def Cmatrix(dim, bound = (5,50)):
    
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        if i == 0:
            j = []
            j.append(0)
            matrix.append(j)
        else:
            j=[]
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.uniform(bound[0], bound[1]))
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M


In [76]:
# Network graph from CI book
D = 70
bounds = (10,1000)
SACOCI = Cmatrix(D, bounds)

ants = 70
alpha = 1
beta = 3
rho = 0.5
runs = 100

ACO101(SACOCI, ants, alpha, beta, rho, runs)

Minimum Cost Found =  3373.0502725552105 Maximum Cost Found =  7269.975462367653


(array([[32, 51, 54, ..., 68, 38, 50],
        [32, 13, 65, ..., 66, 16, 40],
        [32, 51, 57, ..., 47, 44, 28],
        ...,
        [32, 46, 18, ..., 27, 66, 62],
        [32, 13, 65, ..., 21, 69, 15],
        [32, 51, 57, ..., 62, 54, 40]]),
 array([[2.94323498e-26, 5.76880644e-21, 2.94323498e-26, ...,
         7.86449028e-03, 1.12478075e-02, 2.24748157e-02],
        [5.76880644e-21, 2.94323498e-26, 3.89264252e-26, ...,
         2.88282050e-17, 3.72101131e-03, 8.15485078e-02],
        [2.94323498e-26, 3.89264252e-26, 2.94323498e-26, ...,
         2.94323498e-26, 6.84807172e-16, 8.71445188e-15],
        ...,
        [7.86449028e-03, 2.88282050e-17, 2.94323498e-26, ...,
         2.94323498e-26, 4.86511002e-04, 7.97052152e-06],
        [1.12478075e-02, 3.72101131e-03, 6.84807172e-16, ...,
         4.86511002e-04, 2.94323498e-26, 2.30625597e-02],
        [2.24748157e-02, 8.15485078e-02, 8.71445188e-15, ...,
         7.97052152e-06, 2.30625597e-02, 2.94323498e-26]]))

In [98]:
# TSP problem

In [77]:
import RouteMatrix as RM
path = './Data/att48.txt'
SACOCI = RM.TSRM(path)


ants = 48
alpha = 1
beta = 4
rho = 0.5
runs = 100

ACO101(SACOCI, ants, alpha, beta, rho, runs)

Minimum Cost Found =  35682.108919332124 Maximum Cost Found =  48816.15904217578


(array([[13, 24, 12, ...,  0,  8, 39],
        [13, 24, 12, ..., 26, 42, 16],
        [13, 22, 10, ..., 42, 16, 45],
        ...,
        [13, 24, 12, ..., 45, 14, 11],
        [13, 24, 12, ..., 22, 10, 11],
        [13, 24, 12, ..., 16, 39,  7]]),
 array([[1.34377073e-25, 1.57347325e-02, 2.74579085e-02, ...,
         8.59129196e-10, 2.81074882e-03, 1.34377073e-25],
        [1.57347325e-02, 1.34377073e-25, 4.26574910e-03, ...,
         1.72035006e-06, 6.05646279e-03, 7.50036627e-04],
        [2.74579085e-02, 4.26574910e-03, 1.34377073e-25, ...,
         4.27097440e-02, 3.16139573e-02, 1.20315594e-06],
        ...,
        [8.59129196e-10, 1.72035006e-06, 4.27097440e-02, ...,
         1.34377073e-25, 4.23590778e-02, 1.30224362e-05],
        [2.81074882e-03, 6.05646279e-03, 3.16139573e-02, ...,
         4.23590778e-02, 1.34377073e-25, 1.98314162e-25],
        [1.34377073e-25, 7.50036627e-04, 1.20315594e-06, ...,
         1.30224362e-05, 1.98314162e-25, 1.34377073e-25]]))

In [83]:
import RouteMatrix as RM
path = './Data/burma14.txt'
SACOCI = RM.TSRM(path)


ants = 14
alpha = 1
beta = 4
rho = 0.5
runs = 2000

ACO101(SACOCI, ants, alpha, beta, rho, runs)

Minimum Cost Found =  36.00323896455005 Maximum Cost Found =  37.672006667794115


(array([[12,  6,  5, 11, 13,  2,  3,  4,  9,  8, 10,  7,  0,  1],
        [12,  6,  5, 11, 13,  2,  3,  4,  0,  7, 10,  8,  9,  1],
        [12,  6, 13,  2,  3, 11,  5,  4,  9,  8, 10,  1,  0,  7],
        [12,  6, 10,  8,  9,  7,  0,  1, 13, 11,  5,  3,  2,  4],
        [12,  6,  5, 11, 13,  2,  3,  4,  9,  8, 10,  7,  0,  1],
        [12,  6,  5, 11, 13,  2,  3,  4,  9,  8, 10,  7,  0,  1],
        [12,  6,  5, 11, 13,  2,  3,  4,  9,  8, 10,  7,  0,  1],
        [12,  6,  5, 11, 13,  2,  3,  4,  9,  8, 10,  7,  0,  1],
        [12,  6,  5, 11, 13,  2,  3,  4,  9,  8, 10,  7,  0,  1],
        [12,  6,  5, 11, 13,  2,  3,  4,  0,  7, 10,  8,  9,  1],
        [12,  6,  5, 11, 13,  2,  3,  4,  0,  7, 10,  8,  9,  1],
        [12,  6,  5, 11, 13,  2,  3,  4,  9,  8, 10,  7,  0,  1],
        [12,  6, 13,  2,  3, 11,  5,  4,  9,  8, 10,  7,  0,  1],
        [12,  6,  5, 11, 13,  2,  3,  4,  9,  8, 10,  7,  0,  1]]),
 array([[0.00000000e+00, 8.57455607e-01, 7.43442204e-03, 8.48646357e-06,
 

In [113]:
import RouteMatrix as RM
path = './Data/bays29.txt'
SACOCI = RM.TSRM(path)


ants = 29
alpha = 1
beta = 3
rho = 0.5
runs = 100

ACO101(SACOCI, ants, alpha, beta, rho, runs)

Minimum Cost Found =  10317.813605806834 Maximum Cost Found =  13651.622742453783


(array([[ 7, 26, 23,  0, 27,  5, 11,  8,  4, 20,  1, 19,  9,  3, 14, 17,
         13, 21, 10, 15, 12, 22,  6, 24, 18,  2, 28, 25, 16],
        [ 7, 26, 23,  0, 27, 12,  9, 19,  1, 20,  4, 25, 28,  2, 11,  5,
          8, 21, 13, 17, 14,  3, 10, 18, 24,  6, 22, 15, 16],
        [ 7, 23, 26, 15, 12,  9, 19,  1, 20,  4,  8,  5, 27, 11, 25, 28,
          2,  0, 18, 24,  6, 22, 17, 16, 21, 13, 10,  3, 14],
        [ 7, 23, 26, 15, 12,  0, 27,  5, 11,  8,  4, 20,  1, 19,  9,  3,
         14, 18, 24,  6, 22,  2, 28, 25, 17, 13, 21, 16, 10],
        [ 7, 26, 23,  0, 27,  5,  8,  4, 20,  1, 19,  9, 12, 15, 18, 10,
         21, 13, 17, 14,  3, 16,  2, 28, 25, 11,  6, 24, 22],
        [ 7, 26, 23,  0, 27,  5, 11,  8,  4, 20,  1, 19,  9, 12, 15, 18,
         24,  6, 22, 28, 25, 17, 13, 14,  3, 10, 16, 21,  2],
        [ 7, 26, 23,  0, 27,  5, 11, 20,  1, 19,  9,  3, 14, 17, 13, 21,
         10, 18, 15, 12,  2, 28, 25,  4,  8,  6, 22, 24, 16],
        [ 7, 26, 15, 18, 14,  3,  9, 19,  1, 20,  4,  8